# 函数准备

In [13]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import shutil
import random
import string
from random import choice
random.seed(10
# -*- coding:utf-8 -*
import eval
from mkdir import mk_dir
from bert_format import bset,test_qa
import transfer

# 患者数量

In [14]:
df = pd.read_excel("input/human_label.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['样本量']
df["prediction"] = df["Full"]

df = transfer.trans_num(df, "prediction")
    
eval.multiclass_score(df[["golden"]], df[["prediction"]])

结果: 0.6155268022181146 0.7190127873792753 0.6155268022181146 0.5019995644252624


(0.6155268022181146,
 0.7190127873792753,
 0.6155268022181146,
 0.5019995644252624)

In [16]:
col_ab = "num"
col_name = "样本量"

standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,1)), columns = ["prediction"])
for re in range(1,12):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), axis=0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction"]).T
    prediction_0 = pd.concat((prediction_0, prediction),axis=0)
    
df = pd.read_excel("input/human_label.xlsx", index_col=0)
for i in standard_0.index:
    standard_0.loc[i, "golden"] = df.loc[i,col_name]
    
prediction_0 = transfer.trans_num(prediction_0, "prediction")

    
eval.multiclass_score(standard_0[["golden"]], prediction_0[["prediction"]])
pd.concat([standard_0, prediction_0], axis=1).to_csv("eval/evl_%s/eval.csv" % col_ab)

结果: 0.7908309455587392 0.790455834823434 0.7908309455587392 0.7842508423165312


# 患者年龄

In [21]:
df = pd.read_excel("input/human_label.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['年龄']
df["prediction"] = 'none'

df = transfer.transer_age(df, "prediction")
eval.multiclass_score(df[["golden"]], df[["prediction"]])

In [24]:
col_ab =  "age"
col_name = "年龄"

standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,1)), columns = ["prediction"])
for re in range(1,12):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), axis=0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction"]).T
    prediction_0 = pd.concat((prediction_0, prediction),axis=0)
    
df = pd.read_excel("input/human_label.xlsx", index_col=0)

for i in standard_0.index:
    standard_0.loc[i, "golden"] = df.loc[i,col_name]
    
prediction_0 = transfer.transer_age(prediction_0, "prediction")
    
eval.multiclass_score(standard_0[["golden"]], prediction_0[["prediction"]])
pd.concat([standard_0, prediction_0], axis=1).to_csv("eval/evl_%s/eval.csv" % col_ab)

结果: 0.7657657657657657 0.8194397168620116 0.7657657657657657 0.777637246299699


# 患者性别

In [62]:
df = pd.read_excel("input/human_label.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['患者性别']
df["prediction"] = 'none'

for i in df.index:
    
    dig_list = df.loc[i, '患者性别文本'].lower().split(" ")
    for dig in dig_list:
        if dig in ["man", "men", "boy", "boys","male", "males", 
                    "he", "gentleman", "gentlemen", "his", "him"]:
            clas = "男"
            
        if dig in ["woman", "women", "girl","girls", "female", 
                    "females", "fm", "she", "lady", "ladys", "her"]:
            clas = "女"
            
    else:
        clas = "none"
eval.multiclass_score(df[["golden"]], df[["prediction"]])

结果: 0.5397412199630314 0.29132058452718146 0.5397412199630314 0.37840200775319366


/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.5397412199630314,
 0.29132058452718146,
 0.5397412199630314,
 0.37840200775319366)

In [10]:
col_ab =  "gender"
col_name = "患者性别"

standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,1)), columns = ["prediction"])
for re in range(1,12):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), axis=0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction"]).T
    prediction_0 = pd.concat((prediction_0, prediction),axis=0)
    
df = pd.read_excel("input/human_label.xlsx", index_col=0)

for i in standard_0.index:
    
    standard_0.loc[i, "golden"] = df.loc[i,col_name]    
    dig_list = prediction_0.loc[i, 'prediction'].lower().split(" ")
    
    for dig in dig_list:
        if dig in ["man", "men", "boy", "boys","male", "males", 
                    "he", "gentleman", "gentlemen", "his", "him"]:
            clas = "男"
            
        elif dig in ["woman", "women", "girl","girls", "female", 
                    "females", "fm", "she", "lady", "ladys", "her"]:
            clas = "女"
        else:
            clas = "none"
    
    prediction_0.loc[i, "prediction"] = clas

eval.multiclass_score(standard_0[["golden"]], prediction_0[["prediction"]])
pd.concat([standard_0, prediction_0], axis=1).to_csv("eval/evl_%s/eval.csv" % col_ab)

结果: 0.8953488372093024 0.9016774087319621 0.8953488372093024 0.8970392649339614


# 证据等级

In [44]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df = df.fillna("none")
df['golden'] = df['证据等级']
df["prediction"] = 'none'

for i in df.index:
    
    dig_list = df.loc[i, 'Full'].lower().split(" ")
    
    for dig in dig_list:
        if dig in [
            "autopsy", "biopsy", "biopsies", 
            "tissue", 
            "histopathology", "histology", "histological", "histopathological"]:
            clas = "1：同时具备肺组织病理+病原学证据"
        elif dig in [
            "blood",
            "pleural", "purulent",
            "csf","cerebrospinal"]:
                clas = "3：血及无菌体液（如脓胸液）"
        elif dig in [
            "bronchoscopy","bronchoscopies", "tracheal", "bronchial", "throat",
            "bal", "balf", "bronchoalveolar","lavage"]:
                clas ="4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4"
        elif dig in [
            "aspirate", "aspiration", 
            "secretion", "sputum", "sputa","immunohistochemistry","staining"]:
                clas ="5：气管吸出物或痰/BALF纯培养或优势培养出病原体"
        else:
            clas = "none"
    prediction_0.loc[i, "prediction"] = clas
eval.multiclass_score(df[["golden"]], df[["prediction"]])

FileNotFoundError: [Errno 2] No such file or directory: '/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx'

In [11]:
col_ab =  "evi"
col_name = "证据等级"

standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,1)), columns = ["prediction"])
for re in range(1,12):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), axis=0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction"]).T
    prediction_0 = pd.concat((prediction_0, prediction),axis=0)
    
df = pd.read_excel("input/human_label.xlsx", index_col=0)
for i in standard_0.index:
    
    standard_0.loc[i, "golden"] = df.loc[i,col_name]
    dig_list = prediction_0.loc[i, 'prediction'].lower().split(" ")

    for dig in dig_list:
        
        if dig in [
            "autopsy", "biopsy", "biopsies", 
            "tissue", "postmortem",
            "histopathology", "histology", "histological", "histopathological","histopathologic"]:
            clas = "1：同时具备肺组织病理+病原学证据"
        elif dig in [
            "blood","abscess",
            "pleural", "purulent","fluid","pus",
            "csf","cerebrospinal"]:
                clas = "3：血及无菌体液（如脓胸液）"
        elif dig in [
            "bronchoscopy","bronchoscopies","bronchoscopic", "thoracoscopic","bronchoscope","transbronchoscopy",
            "tracheal", "bronchial", "throat","tbna",
            "bal", "balf", "bronchoalveolar","lavage"]:
                clas ="4：气管镜活检组织培养出病原体，或者支气管肺泡灌洗液（BALF）定量培养>10**4"
        elif dig in [
            "aspirate", "aspiration", "aspirates",
            "secretion", "sputum", "sputa","immunohistochemistry","staining"]:
                clas ="5：气管吸出物或痰/BALF纯培养或优势培养出病原体"
        else:
            clas = "none"
            # print(dig, clas)
    prediction_0.loc[i, "prediction"] = clas

eval.multiclass_score(standard_0[["golden"]], prediction_0[["prediction"]])
pd.concat([standard_0, prediction_0], axis=1).to_csv("eval/evl_%s/eval.csv" % col_ab)

结果: 0.7746741154562383 0.799693604783611 0.7746741154562383 0.7600590086716601


In [ ]:
import os
for i in os.listdir("eval"):
    for j in os.listdir("eval/%s" %i ):
        if "output" in j:
            os.remove("eval/%s/%s/pytorch_model.bin" % (i, j))